In [187]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy import stats
from scipy.stats import shapiro # 정규성 검정
from scipy.stats import ttest_ind, ttest_rel, bartlett
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import sqlite3

## 자치구별 건물 연식 전처리

In [2]:
file_path = "C:/Users/ITSC/Desktop/MiniPrj2/Git/Data/상업업무용(매매)_실거래가_20240317102917.csv"

In [3]:
f = open(file_path, encoding = "utf8")
reader = csv.reader(f)

In [4]:
csv_list = []
for i in reader:
    csv_list.append(i)
    
f.close()

In [53]:
transaction_df = pd.DataFrame(csv_list[15:])

In [54]:
transaction_df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,NO,시군구,유형,지번,도로명,용도지역,건축물주용도,도로조건,전용/연면적(㎡),대지면적(㎡),...,층,매수,매도,계약년월,계약일,지분구분,건축년도,해제사유발생일,거래유형,중개사소재지
1,1,서울특별시 성동구 옥수동,일반,4***,독서당로40길,제3종일반주거,제1종근린생활,-,6.16,11.4600,...,,개인,법인,202401,06,,1990,-,중개거래,서울 강남구
2,2,서울특별시 성동구 옥수동,일반,2***,독서당로,제2종일반주거,제1종근린생활,-,2119.97,638.0000,...,,법인,법인,202401,11,,2020,-,중개거래,서울 강남구
3,3,서울특별시 성동구 옥수동,일반,2***,한림말3길,제2종일반주거,제2종근린생활,12m미만,462.65,193.0000,...,,-,-,202308,10,,2020,-,중개거래,서울 강남구
4,4,서울특별시 성동구 옥수동,집합,1***,한림말길,제3종일반주거,제2종근린생활,25m미만,9.52,,...,,-,-,202305,30,,1998,-,직거래,-


In [55]:
new_columns_dict = {i : transaction_df.iloc[0, i] for i in range(0, 21)}

In [56]:
transaction_df.iloc[0, 1]

'시군구'

In [57]:
new_columns_dict

{0: 'NO',
 1: '시군구',
 2: '유형',
 3: '지번',
 4: '도로명',
 5: '용도지역',
 6: '건축물주용도',
 7: '도로조건',
 8: '전용/연면적(㎡)',
 9: '대지면적(㎡)',
 10: '거래금액(만원)',
 11: '층',
 12: '매수',
 13: '매도',
 14: '계약년월',
 15: '계약일',
 16: '지분구분',
 17: '건축년도',
 18: '해제사유발생일',
 19: '거래유형',
 20: '중개사소재지'}

In [58]:
transaction_df = transaction_df.rename(columns = new_columns_dict).iloc[1:, :]

In [59]:
transaction_df = transaction_df.iloc[:, 1:]

In [60]:
transaction_df = transaction_df.reset_index(drop = True)

In [61]:
transaction_df.head()

,시군구,유형,지번,도로명,용도지역,건축물주용도,도로조건,전용/연면적(㎡),대지면적(㎡),거래금액(만원),층,매수,매도,계약년월,계약일,지분구분,건축년도,해제사유발생일,거래유형,중개사소재지
0,서울특별시 성동구 옥수동,일반,4***,독서당로40길,제3종일반주거,제1종근린생활,-,6.16,11.4600,"24,000",,개인,법인,202401,06,,1990,-,중개거래,서울 강남구
1,서울특별시 성동구 옥수동,일반,2***,독서당로,제2종일반주거,제1종근린생활,-,2119.97,638.0000,"2,292,445",,법인,법인,202401,11,,2020,-,중개거래,서울 강남구
2,서울특별시 성동구 옥수동,일반,2***,한림말3길,제2종일반주거,제2종근린생활,12m미만,462.65,193.0000,"550,000",,-,-,202308,10,,2020,-,중개거래,서울 강남구
3,서울특별시 성동구 옥수동,집합,1***,한림말길,제3종일반주거,제2종근린생활,25m미만,9.52,,"22,000",,-,-,202305,30,,1998,-,직거래,-
4,서울특별시 성동구 옥수동,집합,1***,한림말길,제3종일반주거,제2종근린생활,25m미만,9.52,,"24,000",,-,-,202307,13,,1998,-,직거래,-


In [62]:
transaction_df2 = transaction_df.drop(['유형', '지번', '도로명', '도로조건', '전용/연면적(㎡)',
       '대지면적(㎡)', '거래금액(만원)', '층', '매수', '매도', '해제사유발생일', '계약일', '지분구분', '거래유형', '중개사소재지'], axis = 1)

In [63]:
transaction_df2.head()

,시군구,용도지역,건축물주용도,계약년월,건축년도
0,서울특별시 성동구 옥수동,제3종일반주거,제1종근린생활,202401,1990
1,서울특별시 성동구 옥수동,제2종일반주거,제1종근린생활,202401,2020
2,서울특별시 성동구 옥수동,제2종일반주거,제2종근린생활,202308,2020
3,서울특별시 성동구 옥수동,제3종일반주거,제2종근린생활,202305,1998
4,서울특별시 성동구 옥수동,제3종일반주거,제2종근린생활,202307,1998


In [69]:
transaction_df2["시군구"].str.split().map(lambda x: x[1])

0       성동구
1       성동구
2       성동구
3       성동구
4       성동구
       ... 
8441    강동구
8442    강동구
8443    강동구
8444    강동구
8445    강동구
Name: 시군구, Length: 8446, dtype: object

In [82]:
transaction_df2["자치구"] = transaction_df2["시군구"].str.split().map(lambda x: x[1])

In [85]:
transaction_df2.head()

,시군구,용도지역,건축물주용도,계약년월,건축년도,자치구
0,서울특별시 성동구 옥수동,제3종일반주거,제1종근린생활,202401,1990,성동구
1,서울특별시 성동구 옥수동,제2종일반주거,제1종근린생활,202401,2020,성동구
2,서울특별시 성동구 옥수동,제2종일반주거,제2종근린생활,202308,2020,성동구
3,서울특별시 성동구 옥수동,제3종일반주거,제2종근린생활,202305,1998,성동구
4,서울특별시 성동구 옥수동,제3종일반주거,제2종근린생활,202307,1998,성동구


In [97]:
transaction_df2["건축년도"] = transaction_df2["건축년도"].map(lambda x: int(x) if x != "" else 0)

In [100]:
transaction_df2["연식"] = 2024 - transaction_df2["건축년도"]

In [101]:
transaction_df2.head()

,시군구,용도지역,건축물주용도,계약년월,건축년도,자치구,연식
0,서울특별시 성동구 옥수동,제3종일반주거,제1종근린생활,202401,1990,성동구,34
1,서울특별시 성동구 옥수동,제2종일반주거,제1종근린생활,202401,2020,성동구,4
2,서울특별시 성동구 옥수동,제2종일반주거,제2종근린생활,202308,2020,성동구,4
3,서울특별시 성동구 옥수동,제3종일반주거,제2종근린생활,202305,1998,성동구,26
4,서울특별시 성동구 옥수동,제3종일반주거,제2종근린생활,202307,1998,성동구,26


In [107]:
transaction_df2.iloc[:, -2:].loc[transaction_df2["연식"] != 2024].groupby("자치구").mean()

,연식
자치구,
강남구,25.407947
강동구,23.024331
강북구,19.931624
강서구,11.732743
관악구,22.330317
광진구,25.179310
구로구,25.361472
금천구,15.666667
노원구,26.982955


In [171]:
transaction_df3 = transaction_df2.iloc[:, -2:].loc[transaction_df2["연식"] != 2024].groupby("자치구").mean()

In [114]:
ttest_ind(transaction_df2["연식"].loc[transaction_df2["자치구"] == "강남구"], transaction_df2["연식"].loc[transaction_df2["자치구"] == "강동구"], equal_var=False)

TtestResult(statistic=2.2289681602747953, pvalue=0.026100215133744775, df=779.8627034518172)

In [118]:
for i in transaction_df2["자치구"].unique():
    print(f"transaction_df2['연식'].loc[transaction_df2['자치구'] == '{i}']")

transaction_df2['연식'].loc[transaction_df2['자치구'] == '성동구']
transaction_df2['연식'].loc[transaction_df2['자치구'] == '종로구']
transaction_df2['연식'].loc[transaction_df2['자치구'] == '중구']
transaction_df2['연식'].loc[transaction_df2['자치구'] == '용산구']
transaction_df2['연식'].loc[transaction_df2['자치구'] == '광진구']
transaction_df2['연식'].loc[transaction_df2['자치구'] == '동대문구']
transaction_df2['연식'].loc[transaction_df2['자치구'] == '중랑구']
transaction_df2['연식'].loc[transaction_df2['자치구'] == '성북구']
transaction_df2['연식'].loc[transaction_df2['자치구'] == '강북구']
transaction_df2['연식'].loc[transaction_df2['자치구'] == '도봉구']
transaction_df2['연식'].loc[transaction_df2['자치구'] == '노원구']
transaction_df2['연식'].loc[transaction_df2['자치구'] == '은평구']
transaction_df2['연식'].loc[transaction_df2['자치구'] == '서대문구']
transaction_df2['연식'].loc[transaction_df2['자치구'] == '마포구']
transaction_df2['연식'].loc[transaction_df2['자치구'] == '양천구']
transaction_df2['연식'].loc[transaction_df2['자치구'] == '강서구']
transaction_df2['연식'].loc[transaction_df2['자치구'] == '구로

In [136]:
stats.f_oneway(transaction_df2['연식'].loc[transaction_df2['자치구'] == '성동구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '종로구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '중구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '용산구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '광진구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '동대문구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '중랑구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '성북구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '강북구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '도봉구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '노원구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '은평구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '서대문구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '마포구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '양천구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '강서구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '구로구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '금천구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '영등포구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '송파구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '동작구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '관악구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '서초구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '강남구'],
               transaction_df2['연식'].loc[transaction_df2['자치구'] == '강동구'])

F_onewayResult(statistic=20.909518313030752, pvalue=6.48145250281766e-17)

In [ ]:
["transaction_df2['연식'].loc[transaction_df2['자치구'] == '성동구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '종로구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '중구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '용산구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '광진구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '동대문구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '중랑구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '성북구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '강북구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '도봉구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '노원구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '은평구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '서대문구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '마포구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '양천구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '강서구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '구로구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '금천구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '영등포구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '송파구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '동작구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '관악구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '서초구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '강남구']",
 "transaction_df2['연식'].loc[transaction_df2['자치구'] == '강동구']"]

In [137]:
seoul_district = pd.read_csv("C:/Users/ITSC/Desktop/MiniPrj2/Git/Data/seoul_district.csv")

In [141]:
seoul_district = seoul_district.iloc[:, 1:]

In [142]:
seoul_district

,자치구,중대형 지하1층,중대형 1층,중대형 2층,중대형 3층,중대형 4층,중대형 5층,중대형 6-10층,소형 지하1층,소형 1층,소형 2층,면적 (km²)
0,강남구,22.13,64.54,32.00,25.54,23.36,22.50,21.64,29.43,64.67,33.71,39.50
1,강동구,11.10,47.20,20.50,15.20,13.50,12.00,12.10,0.00,56.10,24.70,24.59
2,강북구,13.17,45.93,18.90,15.43,12.50,12.93,14.33,13.00,42.60,17.80,23.60
3,강서구,10.70,35.20,16.30,14.80,12.30,10.10,9.00,6.20,28.70,13.60,41.45
4,관악구,18.40,72.80,28.90,21.95,19.65,21.25,18.05,13.80,65.75,30.85,29.57
5,광진구,14.30,52.55,22.80,16.45,15.25,12.95,13.70,16.05,39.40,20.75,17.06
6,구로구,11.70,45.20,18.80,13.00,11.70,9.10,0.00,0.00,47.10,28.30,20.12
7,금천구,11.70,33.20,14.10,10.60,9.30,11.40,0.00,0.00,31.20,13.20,13.02
8,노원구,8.40,29.60,14.40,12.20,10.90,12.10,0.00,10.00,26.00,9.80,35.44
9,동대문구,11.38,44.32,18.10,13.70,12.58,12.10,7.70,8.72,42.40,19.38,14.22


In [150]:
combined_df = pd.merge(seoul_district, transaction_df3, on = "자치구", how = "left")

In [153]:
combined_df2 = combined_df.iloc[:, [0, 2, 9, 12]]

In [156]:
combined_df2.iloc[:, 1:].corr()

,중대형 1층,소형 1층,연식
중대형 1층,1.000000,0.864503,0.314326
소형 1층,0.864503,1.000000,0.360416
연식,0.314326,0.360416,1.000000


In [157]:
combined_df2.iloc[:, 1:]

,중대형 1층,소형 1층,연식
0,64.54,64.67,25.407947
1,47.20,56.10,23.024331
2,45.93,42.60,19.931624
3,35.20,28.70,11.732743
4,72.80,65.75,22.330317
5,52.55,39.40,25.179310
6,45.20,47.10,25.361472
7,33.20,31.20,15.666667
8,29.60,26.00,26.982955
9,44.32,42.40,30.309764


In [166]:
transaction_df3 = transaction_df3.reset_index().reset_index()

In [174]:
transaction_df3 = transaction_df3.reset_index()

In [180]:
transaction_df3 = transaction_df3.loc[transaction_df3["자치구"].isin(seoul_district["자치구"])].reset_index(drop = True).reset_index()

In [181]:
transaction_df3["index"] = transaction_df3["index"] + 1

In [184]:
transaction_df3["연식"] = round(transaction_df3["연식"], 2)

In [196]:
transaction_df3 = transaction_df3.iloc[:, [0, 2]]

In [199]:
insert_query = """INSERT INTO analysis_data_districtagedata("district_id_id", "district_age") VALUES(?, ?)"""
for i in range(0,len(transaction_df3)):
    data = tuple(transaction_df3.iloc[i, :])
    
    conn = sqlite3.connect("C:/Users/ITSC/Desktop/MiniPrj2/Git/web/db.sqlite3")
    cur = conn.cursor()
    cur.execute(insert_query, data)
    cur.close()
    conn.commit()
    conn.close()

In [192]:
tuple(transaction_df3.iloc[0, [0, 2]])

(1, 25.41)

In [194]:
transaction_df3.dtypes

index      int64
자치구       object
연식       float64
dtype: object

In [197]:
transaction_df3

,index,연식
0,1,25.41
1,2,23.02
2,3,19.93
3,4,11.73
4,5,22.33
5,6,25.18
6,7,25.36
7,8,15.67
8,9,26.98
9,10,30.31
